## Preprocessing

In [95]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import seaborn as sns 

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [96]:
len(application_df)

34299

In [97]:
application_df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [98]:
application_df.describe().transpose

<bound method DataFrame.transpose of                 EIN        STATUS       ASK_AMT  IS_SUCCESSFUL
count  3.429900e+04  34299.000000  3.429900e+04   34299.000000
mean   5.191852e+08      0.999854  2.769199e+06       0.532406
std    2.451472e+08      0.012073  8.713045e+07       0.498956
min    1.052060e+07      0.000000  5.000000e+03       0.000000
25%    2.748482e+08      1.000000  5.000000e+03       0.000000
50%    4.656317e+08      1.000000  5.000000e+03       1.000000
75%    7.526117e+08      1.000000  7.742000e+03       1.000000
max    9.960869e+08      1.000000  8.597806e+09       1.000000>

In [99]:
application_df.corr(numeric_only=True)['IS_SUCCESSFUL'].sort_values()

EIN             -0.044482
ASK_AMT         -0.010861
STATUS          -0.001636
IS_SUCCESSFUL    1.000000
Name: IS_SUCCESSFUL, dtype: float64

In [100]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1,inplace=True)

In [101]:
# Determine the number of unique values in each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [102]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [103]:
#Function to bin the data for APPLICATION_TYPE and CLASSIFICATION
def binning(df, param, value):
    # Look at parameter value counts for binning
    before_counts = df[param].value_counts()
    
    # Choose a cutoff value and create a list of parameter to be replaced
    types_to_replace = list(before_counts[before_counts < value].index)
    
    # Replace in dataframe
    for data in types_to_replace:
        df[param] = df[param].replace(data,"Other")
        
    # Check to make sure binning was successful
    after_counts = df[param].value_counts()
    return print(f" Value Count before binning:{before_counts}Value Count after binning:{after_counts}")

In [104]:
binning (application_df,'APPLICATION_TYPE',500)

 Value Count before binning:APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64Value Count after binning:APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [105]:
# Look at CLASSIFICATION value counts for binning
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x: x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [106]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
binning(application_df,'CLASSIFICATION',800)

 Value Count before binning:CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64Value Count after binning:CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [107]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)
application_numeric

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False


In [108]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)

In [109]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Set features
input_features = len(X_train_scaled[0])

## Compile, Train and Evaluate the Model

In [82]:
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=input_features))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/50


804/804 [==============================] - 3s 2ms/step - loss: 0.5896 - accuracy: 0.7074
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5601 - accuracy: 0.7284
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5562 - accuracy: 0.7303
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7305
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7321
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7320
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7324
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7320
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7324
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7

# Step 3: Optimize the Model

In [110]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice("activation",["relu","tanh","sigmoid"])
    
    # allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int("first_units",
        min_value=1,
        max_value=320,
        step=5), activation=activation, input_dim=input_features))

    # allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int("num_layers", 1, 8)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int("units_" + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return nn_model



In [111]:
# import the kerastuner library
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [112]:
# run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=35,validation_data=(X_test_scaled,y_test)) 

Trial 60 Complete [00h 00m 46s]
val_accuracy: 0.7337609529495239

Best val_accuracy So Far: 0.7337609529495239
Total elapsed time: 00h 22m 24s


In [113]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 86, 'num_layers': 2, 'units_0': 66, 'units_1': 56, 'units_2': 96, 'units_3': 86, 'units_4': 116, 'units_5': 81, 'units_6': 46, 'units_7': 31, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 316, 'num_layers': 5, 'units_0': 46, 'units_1': 41, 'units_2': 31, 'units_3': 111, 'units_4': 56, 'units_5': 101, 'units_6': 21, 'units_7': 51, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 61, 'num_layers': 2, 'units_0': 66, 'units_1': 31, 'units_2': 31, 'units_3': 21, 'units_4': 1, 'units_5': 71, 'units_6': 71, 'units_7': 51, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0023'}


In [114]:
# evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5528 - accuracy: 0.7338 - 777ms/epoch - 3ms/step
Loss: 0.5527785420417786, Accuracy: 0.7337609529495239
268/268 - 1s - loss: 0.5506 - accuracy: 0.7319 - 607ms/epoch - 2ms/step
Loss: 0.5505659580230713, Accuracy: 0.7318950295448303
268/268 - 0s - loss: 0.5482 - accuracy: 0.7317 - 481ms/epoch - 2ms/step
Loss: 0.5482308268547058, Accuracy: 0.7316617965698242


In [115]:
# first best model 
best_model_1 = top_model[0]
best_model_1.build()
best_model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 86)                3784      
                                                                 
 dense_1 (Dense)             (None, 66)                5742      
                                                                 
 dense_2 (Dense)             (None, 56)                3752      
                                                                 
 dense_3 (Dense)             (None, 1)                 57        
                                                                 
Total params: 13335 (52.09 KB)
Trainable params: 13335 (52.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [116]:
# second best model 
best_model_2 = top_model[1]
best_model_2.build()
best_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 316)               13904     
                                                                 
 dense_1 (Dense)             (None, 46)                14582     
                                                                 
 dense_2 (Dense)             (None, 41)                1927      
                                                                 
 dense_3 (Dense)             (None, 31)                1302      
                                                                 
 dense_4 (Dense)             (None, 111)               3552      
                                                                 
 dense_5 (Dense)             (None, 56)                6272      
                                                                 
 dense_6 (Dense)             (None, 1)                 5

In [117]:
# third best model 
best_model_3 = top_model[2]
best_model_3.build()
best_model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 61)                2684      
                                                                 
 dense_1 (Dense)             (None, 66)                4092      
                                                                 
 dense_2 (Dense)             (None, 31)                2077      
                                                                 
 dense_3 (Dense)             (None, 1)                 32        
                                                                 
Total params: 8885 (34.71 KB)
Trainable params: 8885 (34.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [118]:
# Export our model to HDF5 file
best_model_3.save("AlphabetSoupCharity_Optimization.h5")

C:\Users\kaiva\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
